# Create unified catalog
Update of the 31/07/2023 : author [tomsail](https://github.com/tomsail) (Thomas Saillour)

In [ ]:
import pandas as pd
import geopandas as gp
import numpy as np
import matplotlib.pyplot as plt
import shapely

In [ ]:
%matplotlib widget

## COOPS

In [ ]:
from searvey.coops import coops_stations

coops = coops_stations()
coops

In [ ]:
### get lat/lon
coops['lon'] = coops['geometry'].x
coops['lat'] = coops['geometry'].y
coops['nos_id'] = coops.index.astype(int).astype(str)

In [ ]:
coops = coops.drop('geometry',axis=1)
coops

## IOC

In [ ]:
from searvey import ioc
ioc_stations = ioc.get_ioc_stations()
ioc_stations['lon'] = ioc_stations['geometry'].x
ioc_stations['lat'] = ioc_stations['geometry'].y
ioc_stations = ioc_stations.drop('geometry',axis=1)
# dropping all duplicates 
ioc_stations = ioc_stations.drop_duplicates(subset=['ioc_code'])
ioc_stations

## EMODNET

In [ ]:
emodnet = pd.read_csv('emodnet.csv')
emodnet = emodnet.drop_duplicates(['EP_PLATFORM_CODE'])
emodnet[~emodnet.EP_PLATFORM_ID.isna()]

## CMEMS

In [ ]:
cmems = pd.read_csv('cmems.csv')
cmems = cmems.drop(cmems.columns[0],axis=1)
cmems = cmems.drop_duplicates()

## GESLA
### Initialize a GeslaDataset object
First clone the following repository: `https://github.com/philiprt/GeslaDataset`.
Also download the whole GESLA3 dataset at this url: `https://gesla787883612.wordpress.com/downloads/`. 

Then place the `gesla.py` file in your working directory (or elsewhere on your path), and import the `GeslaDataset` class. Selecting and loading data files requires paths to the metadata .csv file and the directory containing the data files. Initialize a `GeslaDataset` object with these paths as follows.

In [ ]:
meta_file = "./GESLA3_ALL.csv"
data_path = "GESLA3.0_ALL/"
from gesla import GeslaDataset

g3 = GeslaDataset(meta_file=meta_file, data_path=data_path)

For possible future sources we can use gesla sources

In [ ]:
g3.meta.contributor_website.unique()
g3.meta

### normalize name, lon, lat

In [ ]:
coops = coops.rename(columns={'name':'Station_Name','lon':'longitude','lat':'latitude'})
coops['coops_id'] = coops['Station_Name']

In [ ]:
ioc_stations = ioc_stations.rename(columns={'location':'Station_Name','lon':'longitude','lat':'latitude'})
ioc_stations['ioc_id'] = ioc_stations['Station_Name'] 

In [ ]:
cmems = cmems.rename(columns={'PLATFORM_NAME':'Station_Name','longitude (degrees_east)':'longitude','latitude (degrees_north)':'latitude'})
cmems['cmems_id'] = cmems['Station_Name'] 

In [ ]:
emodnet = emodnet.rename(columns={'EP_PLATFORM_CODE':'Station_Name','longitude (degrees_east)':'longitude','latitude (degrees_north)':'latitude'})
emodnet['emodnet_id'] = emodnet['Station_Name'] 

In [ ]:
gesla3 = g3.meta.rename(columns={'site_name':'Station_Name','longitude':'longitude','latitude':'latitude'})
gesla3['gesla3_id'] = gesla3['Station_Name']

## Plot

In [ ]:
cmems.plot.scatter(x = 'longitude', y = 'latitude', marker='o', label='cmems')
a = plt.gca()
emodnet.plot.scatter(ax=a, x = 'longitude', y = 'latitude', color='orange', marker='1', label='emodnet')
ioc_stations.plot.scatter(ax=a, x = 'longitude', y = 'latitude', color='green', marker='x', label='ioc')
coops.plot.scatter(ax=a, x = 'longitude', y = 'latitude', color='cyan', marker='+', label='coops')
gesla3.plot.scatter(ax=a, x = 'longitude', y = 'latitude', color='red', marker='.',label='gesla')
plt.legend(loc='upper center',bbox_to_anchor=(.5, 1.1), ncol=5)

## Merge COOPS & ioc_stations

#### check duplicates

In [ ]:
ioc_ = ioc_stations.loc[ioc_stations.country=='USA']
ioc_

In [ ]:
ioc_.country.unique()

In [ ]:
m1 = pd.merge(ioc_,coops, on=['Station_Name']).Station_Name

In [ ]:
m1

In [ ]:
i1 = ioc_.loc[ioc_.Station_Name.isin(m1)]
i1 = i1.drop_duplicates(['Station_Name']).reset_index(drop=True)
i1 = i1.sort_values('Station_Name').reset_index(drop=True)
i1

In [ ]:
g1 = coops.loc[coops.Station_Name.isin(m1)]
g1 = g1.sort_values('Station_Name').reset_index(drop=True)
g1

### Lat/Lon differences

In [ ]:
np.abs(i1.longitude - g1.longitude).max()

In [ ]:
np.abs(i1.latitude - g1.latitude).max()

In [ ]:
ioc_stations.plot.scatter(x = 'longitude', y = 'latitude', color='green', marker='x', label='ioc')
a = plt.gca()
coops.plot.scatter(ax=a, x = 'longitude', y = 'latitude', color='cyan', marker='+', label='coops')

plt.scatter(x=i1.longitude, y=i1.latitude, s=80, facecolors='none', edgecolors='r', label='common')
plt.legend(loc='upper center',bbox_to_anchor=(.5, 1.1), ncol=10)
plt.tight_layout()

In [ ]:
unique = coops[~coops.Station_Name.isin(m1)]
unique

#### normalize country column

In [ ]:
ioc_stations = ioc_stations.rename(columns={'country':'Country'})

In [ ]:
coops['Country'] = 'United States'

In [ ]:
catalog = pd.concat([ioc_stations, coops], ignore_index=True)
catalog.reset_index(inplace=True,drop=True)
catalog

### test it

In [ ]:
minlat = g1.latitude.min(),
maxlat = g1.latitude.max(),
minlon = g1.longitude.min(),
maxlon = g1.longitude.max(),

In [ ]:
w = catalog.loc[(catalog['longitude'] > minlon) & (catalog['longitude'] < maxlon) & (catalog['latitude'] > minlat) & (catalog['latitude'] < maxlat)]

In [ ]:
w.reset_index(inplace=True, drop=True)

In [ ]:
w

In [ ]:
w.loc[~w.ioc_code.isna()] # these can be downloaded from IOC

In [ ]:
w.loc[~w.nws_id.isna()] # these can be downloaded from COOPS

In [ ]:
w[w.duplicated('Station_Name')] # these can be downloaded from both

#### check duplicates with CMEMS

In [ ]:
d = cmems.Station_Name.isin(catalog.Station_Name)
d.sum()

In [ ]:
db2 = cmems[d].sort_values('Station_Name')
db2 = db2.reset_index(drop=True)
db2 = db2.drop_duplicates('Station_Name')
db2 = db2[~db2.Station_Name.isna()]

In [ ]:
db2_ = catalog.loc[catalog.Station_Name.isin(cmems[d].Station_Name)].sort_values('Station_Name')
db2_ = db2_.drop_duplicates('Station_Name')
db2_ = db2_.reset_index(drop=True)
db2_ = db2_[~db2_.Station_Name.isna()]

### Lat/Lon differences

In [ ]:
np.abs(db2.longitude.values - db2_.longitude.values).max()

In [ ]:
np.abs(db2.latitude.values - db2_.latitude.values).max()

In [ ]:
db2.longitude - db2_.longitude

In [ ]:
db2

In [ ]:
mf = (np.abs(db2.longitude - db2_.longitude)<.1)

In [ ]:
db2[~mf]

In [ ]:
db2_[~mf]

In [ ]:
db2 = db2[mf]
db2_ = db2_[mf]

In [ ]:
ioc_stations.plot.scatter(x = 'longitude', y = 'latitude', color='green', marker='x', label='ioc')
a = plt.gca()
coops.plot.scatter(ax=a, x = 'longitude', y = 'latitude', color='cyan', marker='+', label='coops')
cmems.plot.scatter(ax=a, x = 'longitude', y = 'latitude', marker='o', label='cmems')
plt.scatter(x=db2.longitude, y=db2.latitude, s=80, facecolors='none', edgecolors='r', label='common')
plt.legend(loc='upper center',bbox_to_anchor=(.5, 1.1), ncol=4)

### merge cmems

In [ ]:
catalog = pd.concat([catalog, cmems], ignore_index=True)

catalog.reset_index(inplace=True,drop=True)
catalog

### test IOC and CMEMS data

In [ ]:
cc = catalog.loc[catalog.Station_Name.isin(db2_.Station_Name)]

In [ ]:
test = cc.loc[cc.Station_Name=="Bangor"]
test = test.rename(columns={'longitude':'lon','latitude':'lat','Country':'country','Station_Name':'location'})

#### IOC

In [ ]:
url = "http://www.ioc-sealevelmonitoring.org/bgraph.php?code={}&output=tab&period=30&endtime={}".format('bang','2023-7-30') # use IOC code value
url

In [ ]:
data = ioc.get_ioc_data(
    ioc_metadata=test,
    endtime="2023-08-30",
    period=30,
)
data = data.to_dataframe().reset_index(level= 'ioc_code',drop=True)
data = data.rename(columns={'bub':'slev','lon':'longitude','lat':'latitude','country':'Country','location':'Station_Name'})
data = data['slev']
data

#### CMEMS

In [ ]:
from erddapy import ERDDAP

e = ERDDAP(
  server="https://nrt.cmems-du.eu/erddap",
  protocol="tabledap",
)


In [ ]:
e.dataset_id = 'copernicus_GLO_insitu_nrt_TG'

In [ ]:
e.constraints = {
    "time>=": "2023-07-30 T00:00:00Z",
    "time<=": "2023-08-30 T00:00:00Z",
    "PLATFORM_CODE=": "Bangor",
}


In [ ]:
e.variables = [
    "time",
    "SLEV"
]


In [ ]:
df = e.to_pandas()#low_memory=False)

df['time (UTC)'] = pd.DatetimeIndex(df['time (UTC)'])
df = df.set_index('time (UTC)')

df.index = df.index.tz_convert(None)
df.columns = ['slev']
df['slev'] = df['slev'].apply(pd.to_numeric)
df

In [ ]:
(data['2023-07-01 0:0:0':'2022-07-29 0:0:0'] - df['2023-07-01 0:0:0':'2023-07-30 0:0:0']).max()

In [ ]:
df.plot()
p=plt.gca()
data.plot(ax=p, linestyle='--')
plt.legend(['cmems','ioc'])

#### check duplicates with EMODNET

In [ ]:
de = emodnet.Station_Name.isin(catalog.Station_Name)
de.sum()

### merge emodnet

In [ ]:
catalog = pd.concat([catalog,emodnet], ignore_index=True)

catalog

## Merge GESLA3

In [ ]:
m2 = pd.merge(catalog,gesla3, on=['Station_Name']).Station_Name
m2

In [ ]:
i2 = catalog.loc[catalog.Station_Name.isin(m2)]
i2 = i2.drop_duplicates(['Station_Name']).reset_index(drop=True)
i2 = i2.sort_values('Station_Name').reset_index(drop=True)
i2

In [ ]:
g2 = gesla3.loc[gesla3.Station_Name.isin(m2)]
g2 = g2.sort_values('Station_Name').reset_index(drop=True)
g2

In [ ]:
print(np.abs(i2.longitude - g2.longitude).max())
print(np.abs(i2.latitude - g2.latitude).max())

In [ ]:
catalog.plot.scatter(x = 'longitude', y = 'latitude', color='green', marker='x', label='catalog')
a = plt.gca()
gesla3.plot.scatter(ax=a, x = 'longitude', y = 'latitude', color='cyan', marker='+', label='gesla3')

plt.scatter(x=i2.longitude, y=i2.latitude, s=80, facecolors='none', edgecolors='r', label='common')
plt.legend(loc='upper center',bbox_to_anchor=(.5, 1.1), ncol=10)
plt.tight_layout()

In [ ]:
unique = gesla3[~gesla3.Station_Name.isin(m2)]
unique

In [ ]:
gesla3 = gesla3.rename(columns={'country':'Country'})

In [ ]:
catalog = pd.concat([catalog, gesla3],ignore_index=True)
catalog

In [ ]:
catalog.duplicated('Station_Name').sum()

# Merge all the data together 

In [ ]:
grouped_df = catalog.groupby('Station_Name', as_index=False).agg(lambda x: x.dropna().iloc[0] if x.notna().any() else pd.NA)

In [ ]:
grouped_df

## keep providers info

In [ ]:
provs = ['ioc', 'coops', 'emodnet', 'cmems', 'gesla3']
N_prov = len(provs)

In [ ]:
import itertools
from matplotlib.pyplot import cm
fact = 1
for i in range(1, N_prov):
    fact = fact * i

print(fact, 'combinations of providers possible')

color = iter(cm.tab20(np.linspace(0, 1, 16)))
fix, ax = plt.subplots(figsize=(20,10))
plt.tight_layout()
for ii in range(5)[1:]:
    combs = itertools.combinations(provs, ii)
    for comb in list(combs):
        subset_id = [p + '_id' for p in list(comb)]
        subset_df = grouped_df.dropna(how="any", subset=subset_id)
        if len(subset_df) > 0:
            c = next(color).reshape(1,-1)
            subset_df.plot.scatter(ax=ax, x='longitude',y='latitude',label='-'.join(comb),color=c)
ax.legend()

In [ ]:
grouped_df = grouped_df.sort_values(['Country', 'Station_Name', 'longitude', 'latitude']).reset_index(drop=True)
grouped_df['seaset_id'] = grouped_df.index

In [ ]:
grouped_df.to_csv('catalog_full.csv')
grouped_df

## Minimize the dataset and regroup stations at the same location

In [ ]:
grouped_df.duplicated(['latitude','longitude']).sum() #there are duplicates

In [ ]:
dmask = grouped_df.duplicated(['latitude','longitude'], keep = False)

In [ ]:
s1 = grouped_df.drop_duplicates(['latitude','longitude'])

In [ ]:
s2 = s1.sort_values(['latitude','longitude'])

In [ ]:
ds = s2[['latitude','longitude']].diff() < [0.02,0.02]

In [ ]:
ds_ = ds.all(axis='columns') # get where you match both lat/lon

In [ ]:
s2[ds_ | ds_.shift(-1)]

In [ ]:
dps = s2[ds_ | ds_.shift(-1)].index.tolist()[::2] #find which indices to drop

In [ ]:
s3 = s2.drop(dps)

In [ ]:
s3.duplicated('Station_Name').sum()
s3

In [ ]:
s3.to_csv('catalog_minimal.csv')